### Initialize libraries

In [116]:
%matplotlib inline

from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import PatchCollection
from scipy.spatial.distance import cdist

from ipywidgets import interact, HTML, FloatSlider
from IPython.display import clear_output, display

import numba as nb
import time

### Initialize Model Constants

In [117]:
RJupiter = 6.99e9    # cm

gamma = 2
G = 6.67e-8    # dyne*cm^2/g^2

rhoC = 5    # g/cm^3, central density
PC = 6.5e13    # dyne/cm^2
TC = 22000    # K

K = 2.6e12    # dyne*cm^4/g^2

alpha = np.sqrt(K/(2*np.pi*G))

h = 1e9      

### Initialize Radial Position of Planets

In [118]:
N1 = 1000        # Particles in planet 1

# Use partition to give initial radial positions of particles
partitionNum = 5     
rSpace = np.linspace(0, RJupiter, partitionNum)
zetaSpace = rSpace/alpha

# Establish number of particles in each region of delta(zeta)
NDistribution = []    
for i in range(1,len(zetaSpace)):
    zeta2 = zetaSpace[i]
    zeta1 = zetaSpace[i-1]
    NDistribution.append((np.sin(zeta2) - zeta2*np.cos(zeta2) - np.sin(zeta1) + zeta1*np.cos(zeta1))\
                         *N1/np.pi)
    
NDistribution = np.array(NDistribution)
NDistribution = np.round(NDistribution)

# Create radial distribution
radiusDistribution = []
i = 0
for N in NDistribution:
    radiusDistribution.append(np.random.uniform(rSpace[i], rSpace[i+1], size=N))
    i += 1
    
# Flatten radial array
radiusDistribution = [item for sublist in radiusDistribution for item in sublist]
radiusDistribution = np.array(radiusDistribution)

# Create angle distribution
thetaDistribution = np.random.uniform(0, 2*np.pi, size=len(radiusDistribution))

c:\users\oromi_000\anaconda\lib\site-packages\ipykernel\__main__.py:22: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### Initialize Cartesian Position of Planets

In [119]:
def polar2cart(r, theta):
    return np.array([r*np.cos(theta), r*np.sin(theta)]).T

xyDistribution = polar2cart(radiusDistribution, thetaDistribution)
nParticles = len(xyDistribution)

### Model Initial Velocity & Mass 

In [120]:
velocityDistribution = np.zeros_like(xyDistribution)
MJupiter = 1.89e30    # grams
mDistribution = np.ones(len(xyDistribution)) * MJupiter/len(xyDistribution)

# Helper Functions
### Smoothing function

In [121]:
def W(dist, h):
    '''
    Inputs:
        dist: a scalar distance between particles i an j
        h: smoothing length
    '''
    if dist < h:
        return 10/(7*np.pi*h**2) * \
               (1/4*(2-dist/h)**3 - (1-dist/h)**3)
    elif dist > 2 * h:
        return 0
    else:
        return 10/(7*np.pi*h**2) * (1/4*(2-dist/h)**3)

In [122]:
def gradW(xyDist, h):
    '''
    Inputs:
        xyDist: a [2,1] array containing the difference in [x,y] position
            between two particles
        h: smoothing length
    Outputs:
        a [2,1] array containing the [x,y] component of gradW
    '''
    
    dist = np.sqrt(xyDist[0]**2 + xyDist[1]**2)
    
    if dist < h:
        return np.array(
              [15*xyDist[0]*(3*dist - 4*h)/(14 * np.pi * h**5),
               15*xyDist[1]*(3*dist - 4*h)/(14 * np.pi * h**5)])
    
    elif dist > 2 * h:
        return np.array([0,0])
    
    else:
        return np.array(
                [-15*xyDist[0]*(2*h - dist)**2/(14*h**5*np.pi*dist),
                 -15*xyDist[1]*(2*h - dist)**2/(14*h**5*np.pi*dist)] )

### Density Update Function

In [123]:
def densityUpdate():
    
    global xyDistribution
    global mDistribution

    rhoDistribution = np.empty_like(mDistribution)
    for i in range(0, nParticles):
        rhoDistribution[i] = mDistribution[i]*W(0, h)

        for j in range(i, nParticles):
            if (True):
                xdist = (xyDistribution[i,0]-xyDistribution[j,0])
                ydist = (xyDistribution[i,1]-xyDistribution[j,1])
            dist_ij = np.sqrt(xdist**2 + ydist**2)
            rho_ij = mDistribution[i]*W(dist_ij, h)
            rhoDistribution[i] += rho_ij
    return rhoDistribution

### Define Pressure Gradient

In [125]:
def gradP():
    
    gradPArray = np.zeros_like(velocityDistribution)

    for i in range(0, nParticles):
        for j in range(i, nParticles):
            
            xdist = (xyDistribution[i,0]-xyDistribution[j,0])
            ydist = (xyDistribution[i,1]-xyDistribution[j,1])
            distArr = np.array([xdist,ydist])
            
            gradPX = mDistribution[i]*((pressureDistribution[i]/(rhoDistribution[i]**2))+(pressureDistribution[j]/(rhoDistribution[j]**2)))*gradW(distArr,h)[0]
            gradPY = mDistribution[i]*((pressureDistribution[i]/(rhoDistribution[i]**2))+(pressureDistribution[j]/(rhoDistribution[j]**2)))*gradW(distArr,h)[1]
            
            gradPArray[i,0] += gradPX
            gradPArray[i,1] += gradPY

    return gradPArray

### Define gravity

In [126]:
def gravity():
    
    global velocityDistribution
    global xyDistribution
    
    deltaV = np.zeros_like(xyDistribution, dtype = np.float)
    for j in range(0, nParticles):
        for k in range(0, nParticles):
            if (k!=j):
                xdist = (xyDistribution[j,0]-xyDistribution[k,0])
                ydist = (xyDistribution[j,1]-xyDistribution[k,1])
                #print(xdist)
                #print(ydist)

                if(xdist==0):
                    deltaV[j,0] += 0
                elif(xdist!=0):
                    deltaV[j,0] += -G*mDistribution[j]*xdist/((np.sqrt(xdist**2+ydist**2))**3)
                    #print("blah", -G*mDistribution[i]/(np.sqrt(abs(position[j,0]-position[i,0]))**2))
                    #print("v", i, "x", velocityDistribution[i,0])

                if(ydist==0):
                    deltaV[j,1] += 0
                elif(ydist!=0):
                    deltaV[j,1] += -G*mDistribution[j]*ydist/((np.sqrt(xdist**2+ydist**2))**3)
                    #print("v", i, "y", velocityDistribution[i,1])
    return deltaV

nb_gravity = nb.autojit(gravity)

# Main
### Run through RK1

In [127]:
# RK1 Parameters
t0 = time.time()
t = 0
dt = 5e0
stepN = 100

# Particle history
ParticlePositionHistory = np.zeros((stepN,nParticles,2))
ParticleVelocityHistory = np.zeros((nParticles,2,stepN))
ParticleRhoHistory = np.zeros((nParticles, stepN))

xyDistributionOld = np.copy(xyDistribution)
# rhoOld = np.copy(rhoDistribution)
rhoOld = densityUpdate()
deltaVf = np.zeros_like(xyDistribution, dtype = np.float)

for i in range(stepN):
    t += 1

    # Update density and pressure
    rhoDistribution = densityUpdate()
    pressureDistribution = K*rhoDistribution**2
    ParticleRhoHistory[:,i] = rhoDistribution
    
    # Update velocity and positions
    ParticlePositionHistory[i,:,:] = xyDistribution
    
    deltaVf += nb_gravity()
    deltaVf += -gradP()/rhoDistribution[:,np.newaxis]
#    deltaVf -= v*velocityDistribution
    velocityDistribution += dt*deltaVf
    #deltaPos = velocityDistribution * dt
    xyDistribution += dt*velocityDistribution
    
### --- Debugging --- ###
#    print "Mean velocity at timestep", t, "\n", np.mean(deltaV)
#    print "deltaV at timestep", t, "\n", deltaVf
#    print "deltaPos at timestep", t, "\n", dt*velocityDistribution
#    print "velocityDistribution at timestep", t, "\n", velocityDistribution

#     print "deltaPos:\n", deltaPos
#     print "xyDistribution:\n", xyDistribution

    

print(time.time()-t0)
#print(rhoDistribution-rhoOld)
#print(xyDistribution-xyDistributionOld) 
#print(mDistribution)

KeyboardInterrupt: 

In [ ]:
slider = FloatSlider(description='Time', min=1, max=stepN, step=1)

def update_plot():
    time=slider.value
    x = ParticlePositionHistory[time-1,:,0]
    y = ParticlePositionHistory[time-1,:,1]
    fig = plt.figure(figsize=(10,10))
    plt.scatter(x, y, c=ParticleRhoHistory[:,time-1])
    plt.xlim(-2e10, 2e10)
    plt.ylim(-2e10, 2e10)
#    plt.colorbar()

    clear_output(True)

slider.on_trait_change(update_plot, 'value')


display(slider)
update_plot()